### Perform sh cmds on multiple devices using netmiko

In [6]:
# --- setup ----
# install using "pip install netmiko"
# Fill in username, password, secret, path for file capture/logging
# Run in conda/ or any python IDE
# ----------------------
# An extension of the multi_dev_shCmds.py. Includes a logging feature
# Creates and logs a file (for capturing the outputs from device/devices)
# Created file contains one or more devices (depending the no. of specified devices)
# Tested success on Nexus7000, ASR1004, C9500, c4500, C9300, c3850
# 1st for loop logs into device; 2nd for loop runs the cmd list for the current device
# Cmd list do not need to include "term len 0" (longer output is parsed automatically by Netmiko)
# Create dictionary "connect_info" and use as input for method ConnectHandler()
# Some cmd list are privilleged, thats why we need to specify secret password and do enable

from netmiko import ConnectHandler
import time

nStartTime = time.time()
ask_for_filename = input ("Enter filename :")

file_path = "C:\\Users\\user1\\scripts\\Jupyter\\" + ask_for_filename
log_file = open(file_path, "w")

dev_list = [
"router1", 
]

cmd_list = [
"sh clock",
"sh ver",
]
 
for dev in range(len(dev_list)):
    connect_info = {
        'device_type': 'cisco_ios',
        'ip': dev_list[dev],
        'username': 'user1',
        'password': 'user1password',
        'secret': 'user1password',
    }

    ssh_session = ConnectHandler(**connect_info)
    ssh_session.enable()
    log_file.write('Connecting to ' + str(dev_list[dev]) + "\n")
    log_file.write('-'*79 + "\n")
    for cmd in range(len(cmd_list)):
        current_cmd_capture = ssh_session.send_command(cmd_list[cmd])
        log_file.write(str(cmd_list[cmd]) + "\n")
        log_file.write(current_cmd_capture + "\n")
    log_file.write('-'*79 + "\n")
 
# Close session after each call
    ssh_session.disconnect()

log_file.close()

nTimeElapsed = time.time() - nStartTime
timeSec = (nTimeElapsed/60)
timeTook = (str(timeSec))[:4]
print(("Time took %s minutes" % timeTook))

Enter filename :test
Time took 0.16 minutes


### Perform sh cmds on multiple devices using netmiko (with password encryption)

In [3]:
# Same functionality but with password encryption using Fernet

from cryptography.fernet import Fernet
from netmiko import ConnectHandler
import time

def pwd_decrypt():
    with open('C:\\Users\\user1\\scripts\\rw\\key.key', 'rb') as file_object:  
        key_read_decrypt = file_object.read()

    cipher = Fernet(key_read_decrypt)
    with open('C:\\Users\\user1\\scripts\\rw\\encPwd.key', 'rb') as file_object:  
        encPwd_contents = file_object.read()

    decPwd = cipher.decrypt(encPwd_contents).decode()
    return decPwd

nStartTime = time.time()
ask_for_filename = input ("Enter filename :")

file_path = "C:\\Users\\user1\\scripts\\Jupyter\\" + ask_for_filename
log_file = open(file_path, "w")

dev_list = [
"router1",
"router2",
]

cmd_list = [
"sh cdp nei deta",
]

pwd = pwd_decrypt()
 
for dev in range(len(dev_list)):
    connect_info = {
        'device_type': 'cisco_ios',
        'ip': dev_list[dev],
        'username': 'user1',
        'password': pwd,
        'secret': pwd,
    }

    ssh_session = ConnectHandler(**connect_info)
    ssh_session.enable()
    log_file.write('Connecting to ' + str(dev_list[dev]) + "\n")
    log_file.write('-'*79 + "\n")
    for cmd in range(len(cmd_list)):
        current_cmd_capture = ssh_session.send_command(cmd_list[cmd])
        log_file.write(str(cmd_list[cmd]) + "\n")
        log_file.write(current_cmd_capture + "\n")
    log_file.write('-'*79 + "\n") 
    ssh_session.disconnect()
log_file.close()

nTimeElapsed = time.time() - nStartTime
timeSec = (nTimeElapsed/60)
timeTook = (str(timeSec))[:4]
print(("Time took %s minutes" % timeTook))

Enter filename :test
Time took 0.27 minutes
